# Public Transit Play I

We define the "Maximum Stations Number" as 

**the minimum number of stations one needs to pass through to reach all other stations in a subway network.**

This code is designed to identify the station with the minimum "Maximum Stations Number" within the Nanjing subway network.

We use The real-world data from Nanjing Railway in Jiangsu Province, China, and Shanghai, China.


In [3]:
import networkx as nx

def build_graph_from_metro_data(metro_network):
    # 创建无向图
    graph = nx.Graph()
    for station, connections in metro_network.items():
        for neighbor, distance in connections.items():
            # graph.add_edge(station, neighbor, weight=distance)
            graph.add_edge(station, neighbor, weight=1)
    return graph

def read_metro_data(file_path):
    from collections import defaultdict

    # 邻接表表示的地铁网络图
    metro_network = defaultdict(dict)

    with open(file_path, 'r', encoding='utf-8') as file:
        current_line = None
        for line in file:
            line = line.strip()
            if line.startswith("Line"):
                # 读取线路名
                current_line = line
            elif line:
                # 读取站点和距离信息
                station1, station2, distance = line.split(",")
                distance = int(distance)  # 转换距离为整数
                # 添加到邻接表，视为无向图
                metro_network[station1][station2] = distance
                metro_network[station2][station1] = distance
                # metro_network[station1][station2] = 1
                # metro_network[station2][station1] = 1

    return metro_network

# 示例调用
file_path = "./data/PublicTransit/NJ.txt"  # 替换为你的实际文件路径
# file_path = "./data/PublicTransit/SH_new.txt"  # 替换为你的实际文件路径
# file_path = "./data/PublicTransit/BJ.txt"  # 替换为你的实际文件路径
metro_network = read_metro_data(file_path)

# 打印部分结果查看结构
for station, connections in list(metro_network.items())[:5]:
    print(f"Station: {station}, Connections: {connections}")
    

def calculate_shortest_paths_for_all_stations(graph):
    # 保存每个站点到其他所有站点的最短距离和路径
    shortest_paths_info = {}

    # 使用 NetworkX 的 all_pairs_dijkstra 函数
    all_pairs_shortest_paths = dict(nx.all_pairs_dijkstra(graph, weight="weight"))

    for start_station, (distances, paths) in all_pairs_shortest_paths.items():
        shortest_paths_info[start_station] = {
            "distances": distances,  # 起点到其他站点的最短距离
            "paths": paths           # 起点到其他站点的最短路径
        }

    return shortest_paths_info

# 示例调用
graph = build_graph_from_metro_data(metro_network)  # 使用之前的 build_graph_from_metro_data 函数
shortest_paths_info = calculate_shortest_paths_for_all_stations(graph)

max_distance_info = {}  # 保存每个站点的最远距离、路径和最多边数信息

for station, info in shortest_paths_info.items():
    distances = info["distances"]
    paths = info["paths"]

    # 计算最远距离和最多边数
    farthest_station = max(distances, key=distances.get)  # 最远距离对应的终点站
    max_distance = distances[farthest_station]           # 最远距离
    max_edges = len(paths[farthest_station]) - 1         # 最多边数（路径长度 - 1）
    max_path = paths[farthest_station]                  # 最远距离对应的路径

    # 保存结果
    max_distance_info[station] = {
        "farthest_station": farthest_station,
        "max_distance": max_distance,
        "max_edges": max_edges,
        "max_path": max_path
    }

# 筛选“到其他所有节点的最远距离最短”的前5个节点
sorted_stations = sorted(
    max_distance_info.items(),
    key=lambda x: x[1]["max_distance"]
)[:5]  # 根据最远距离升序取前5个节点

# 打印筛选结果
print("到其他所有节点的最远距离最短的前5个节点:")
for station, result in sorted_stations:
    print(f"站点 {station}:")
    print(f"  到达最远站点: {result['farthest_station']}")
    print(f"  最远距离: {result['max_distance']} 米")
    print(f"  路径经过最多边数: {result['max_edges']}")
    print(f"  对应路径: {result['max_path']}")
    print()

Station: 八卦洲大桥南, Connections: {'笆斗山': 1370}
Station: 笆斗山, Connections: {'八卦洲大桥南': 1370, '燕子矶': 1292}
Station: 燕子矶, Connections: {'笆斗山': 1292, '吉祥庵': 1115}
Station: 吉祥庵, Connections: {'燕子矶': 1115, '晓庄': 1007}
Station: 晓庄, Connections: {'吉祥庵': 1007, '迈皋桥': 1910, '万寿': 1405, '幕府山': 1996}
到其他所有节点的最远距离最短的前5个节点:
站点 珠江路:
  到达最远站点: 金牛湖
  最远距离: 25 米
  路径经过最多边数: 25
  对应路径: ['珠江路', '鼓楼', '玄武门', '新模范马路', '南京站', '小市', '五塘广场', '上元门', '柳洲东路', '天润城', '泰冯路', '高新开发区', '信息工程大学', '卸甲甸', '大厂', '葛塘', '长芦', '化工园', '六和开发区', '龙池', '雄州', '凤凰山公园', '方州广场', '沈桥', '八百桥', '金牛湖']

站点 大行宫:
  到达最远站点: 金牛湖
  最远距离: 25 米
  路径经过最多边数: 25
  对应路径: ['大行宫', '浮桥', '鸡鸣寺', '南京林业大学新庄', '南京站', '小市', '五塘广场', '上元门', '柳洲东路', '天润城', '泰冯路', '高新开发区', '信息工程大学', '卸甲甸', '大厂', '葛塘', '长芦', '化工园', '六和开发区', '龙池', '雄州', '凤凰山公园', '方州广场', '沈桥', '八百桥', '金牛湖']

站点 浮桥:
  到达最远站点: 无想山
  最远距离: 25 米
  路径经过最多边数: 25
  对应路径: ['浮桥', '大行宫', '常府街', '夫子庙', '武定门', '雨花门', '卡子门', '大明路', '明发广场', '南京南站', '翠屏山', '河海大学·佛城西路', '吉印大道', '正方中路', '翔宇路北', '翔宇路南', '禄口机场', '空港新

In [10]:
metro_network

defaultdict(dict,
            {'八卦洲大桥南': {'笆斗山': 1},
             '笆斗山': {'八卦洲大桥南': 1, '燕子矶': 1},
             '燕子矶': {'笆斗山': 1, '吉祥庵': 1},
             '吉祥庵': {'燕子矶': 1, '晓庄': 1},
             '晓庄': {'吉祥庵': 1, '迈皋桥': 1, '万寿': 1, '幕府山': 1},
             '迈皋桥': {'晓庄': 1, '红山动物园': 1},
             '红山动物园': {'迈皋桥': 1, '南京站': 1},
             '南京站': {'红山动物园': 1, '新模范马路': 1, '小市': 1, '南京林业大学新庄': 1},
             '新模范马路': {'南京站': 1, '玄武门': 1},
             '玄武门': {'新模范马路': 1, '鼓楼': 1},
             '鼓楼': {'玄武门': 1, '珠江路': 1},
             '珠江路': {'鼓楼': 1, '新街口': 1},
             '新街口': {'珠江路': 1, '张府园': 1, '上海路': 1, '大行宫': 1},
             '张府园': {'新街口': 1, '三山街': 1},
             '三山街': {'张府园': 1, '中华门': 1},
             '中华门': {'三山街': 1, '安德门': 1},
             '安德门': {'中华门': 1, '天隆寺': 1, '小行': 1},
             '天隆寺': {'安德门': 1, '软件大道': 1},
             '软件大道': {'天隆寺': 1, '花神庙': 1},
             '花神庙': {'软件大道': 1, '南京南站': 1},
             '南京南站': {'花神庙': 1,
              '双龙大道': 1,
       